# 第二次大作業



根據finance.yahoo.com網站資料，我們可以下載比特幣資料與美股資料，考慮樣本期間`2017-01-01`到`2020-12-15`：

1. 請試著對比特幣與S&P500指數日報酬率做出敘述統計。
    - 平均數、標準差、偏態、峰態
    - 根據上述結果，討論哪種資產CP值較高
    - 根據相關係數，說明比特幣可以用來避險嗎

2. 利用移動平均的方式，對比特幣做以下投資操作。(利用套件`vectorbt`)
    - 建立進場與出場訊號並列出
    - 此操作策略的累積報酬率為何?
    - 此操作策略的年化波動率為何?
    - 此操作策略有比一般策略好嗎?
   
3. 利用移動平均的方式，對S&P500指數做以下投資操作。(利用套件`vectorbt`)
    - 建立進場與出場訊號並列出
    - 此操作策略的累積報酬率為何?
    - 此操作策略的年化波動率為何?
    - 此操作策略有比一般策略好嗎?

4. 利用迴歸分析，討論
    - 比特幣報酬率對於S&P500報酬的預測能力
    - S&P500報酬率對於比特幣報酬的預測能力
    
    
5. [加分題]在不利用套件`vectorbt`下，以第二題為例
    - 根據你的策略下，計算出進場次數與出場次數，解釋這兩種次數之間的關係 (4%)
    - 假設不考慮任何策略，在一開始便進場，最後出場，計算此策略(buy-and-hold)的年化報酬率與年化波動率為何 (4%)
    - 建立進場與出場訊號之後，在不限制資金的情況下，計算出此策略的最終淨利 (=獲利減去成本) (8%)
    - 請在12/31前上moodle完成(Week14)課堂反映問卷 (2%)

### 繳交作業注意事項
1. 檔名統一為 "BHW2_學號_班別_姓名"。例如: `BHW2_08151320_經1C_王大明` (扣2分) 
2. 繳交時間為1/6中午12點前 (扣4分)
3. 繳交方式：Moodle的作業區繳交 (扣2分)
4. 若未依規定繳交，成績折扣如上所示
    

In [1]:
!pip install ffn
!pip install vectorbt
!pip install seaborn
!pip install matplotlib
!pip install numpy
!pip install pandas
!pip install statsmodels
!pip install yfinance
!pip install plotly

In [2]:
%matplotlib inline
import math
import ffn
import plotly.graph_objects
import plotly.subplots
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

import seaborn as sns
sns.set_style("darkgrid")
import vectorbt as vbt

/usr/local/lib/python3.6/dist-packages/ffn/core.py:27: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)


In [3]:
data = ffn.get(["^GSPC", "BTC-USD"], start="2017-01-01", end="2020-12-15")

## 1. 請試著對比特幣與S&P500指數日報酬率做出敘述統計。

  - 平均數、標準差、偏態、峰態
  - 根據上述結果，討論哪種資產CP值較高
  - 根據相關係數，說明比特幣可以用來避險嗎

In [4]:
data = ffn.get(["^GSPC", "BTC-USD"], start="2017-01-01", end="2020-12-15")

data['gspc_ret'] = data.gspc.diff() / data.gspc.shift()
data['btcusd_ret'] = data.btcusd.diff() / data.btcusd.shift()

### 比特幣

In [5]:
mean = data.btcusd_ret.mean()
print(F"比特幣 報酬率平均值：{mean}")
std = data.btcusd_ret.std()
print(F"比特幣 報酬率標準差：{std}")
skew = data.btcusd_ret.skew()
print(F"比特幣 報酬率偏態：{skew}")
kurt = data.btcusd_ret.kurt()
print(F"比特幣 報酬率峰態：{kurt}")
cp = mean / std
print(F"比特幣 CP值：{cp}")

比特幣 報酬率平均值：0.0041740723279642955
比特幣 報酬率標準差：0.04936173911161406
比特幣 報酬率偏態：-0.12573707859528882
比特幣 報酬率峰態：6.342037149974088
比特幣 CP值：0.08456088466668712


### S&P500

In [6]:
mean = data.gspc_ret.mean()
print(F"S&P500 報酬率平均值：{mean}")
std = data.gspc_ret.std()
print(F"S&P500 報酬率標準差：{std}")
skew = data.gspc_ret.skew()
print(F"S&P500 報酬率偏態：{skew}")
kurt = data.gspc_ret.kurt()
print(F"S&P500 報酬率峰態：{kurt}")
cp = mean / std
print(F"S&P500 CP值：{cp}")

S&P500 報酬率平均值：0.0005797769503467628
S&P500 報酬率標準差：0.012973249590967554
S&P500 報酬率偏態：-0.7281322325889282
S&P500 報酬率峰態：19.434759832277464
S&P500 CP值：0.044690187009924214


### 相關係數

In [7]:
corr = data.gspc_ret.corr(data.btcusd_ret)
print(F"比特幣報酬率與 S&P500 報酬率的相關係數：{corr}, 呈現低度正相關")

比特幣報酬率與 S&P500 報酬率的相關係數：0.11383595592029477, 呈現低度正相關


### Result
由於 比特幣的 CP 值比 S&P500 CP 值高了一倍，而且比特幣與美股呈現低度正相關，也就是幾乎沒有關聯性，因此是個很好的避險手段。

## 2. 利用移動平均的方式，對比特幣做以下投資操作。(利用套件vectorbt)

1. 建立進場與出場訊號並列出
2. 此操作策略的累積報酬率為何?
3. 此操作策略的年化波動率為何?
4. 此操作策略有比一般策略好嗎?

In [8]:
data1 = ffn.get(["^GSPC", "BTC-USD"], start="2017-01-01", end="2020-12-15")

maA = data1.btcusd.rolling(3).mean()
maB = data1.btcusd.rolling(5).mean()

buy = (maA>maB)&(maB.shift()>maA.shift())
sell = (maB>maA)&(maA.shift()>maB.shift())

result = vbt.Portfolio.from_signals(data1.btcusd, buy, sell, freq='1D')

# 1. 建立進場與出場訊號並列出
print("=====================================")
print("1. 建立進場與出場訊號並列出")
print(data[buy].btcusd)
print(data[sell].btcusd)

# 2. 此操作策略的累積報酬率為何?
ans = result.cumulative_returns().tail()
print("=====================================")
print(F"2. 累計報酬率：\n{ans*100}%")


# 3. 此操作策略的年化波動率為何?
ans = result.annualized_volatility()
print("=====================================")
print(F"3. 年化波動率：{ans*100}%")

# 4. 此操作策略有比一般策略好嗎?
print("=====================================")
print("4. 從圖三的 Cumulative Returns 來看的話，此策略的收益與一般策略差不多。")
result.plot().show()

1. 建立進場與出場訊號並列出
Date
2017-01-17      907.937988
2017-01-27      919.750000
2017-02-15     1007.479980
2017-03-14     1240.000000
2017-03-29     1026.430054
                  ...     
2020-10-19    11916.334961
2020-11-12    16276.343750
2020-12-02    19201.091797
2020-12-07    19191.630859
2020-12-14    19246.644531
Name: btcusd, Length: 102, dtype: float64
Date
2017-01-26      917.585999
2017-02-10      988.674011
2017-03-08     1150.000000
2017-03-17     1100.229980
2017-04-13     1167.540039
                  ...     
2020-10-16    11358.101562
2020-11-11    15701.339844
2020-11-27    17108.402344
2020-12-04    18699.765625
2020-12-08    18321.144531
Name: btcusd, Length: 101, dtype: float64
2. 累計報酬率：
Date
2020-12-09    1530.78028
2020-12-10    1530.78028
2020-12-11    1530.78028
2020-12-14    1530.78028
2020-12-15    1545.22106
Name: btcusd, dtype: float64%
3. 年化波動率：64.27070650765494%
4. 從圖三的 Cumulative Returns 來看的話，此策略的收益與一般策略差不多。


## 3. 利用移動平均的方式，對 S&P500 做以下投資操作。(利用套件vectorbt)

1. 建立進場與出場訊號並列出
2. 此操作策略的累積報酬率為何?
3. 此操作策略的年化波動率為何?
4. 此操作策略有比一般策略好嗎?


In [9]:
data2 = ffn.get(["^GSPC", "BTC-USD"], start="2017-01-01", end="2020-12-15")

maA = data2.gspc.rolling(3).mean()
maB = data2.gspc.rolling(5).mean()

buy = (maA>maB)&(maB.shift()>maA.shift())
sell = (maB>maA)&(maA.shift()>maB.shift())

result = vbt.Portfolio.from_signals(data1.gspc, buy, sell, freq='1D')

# 1. 建立進場與出場訊號並列出
print("=====================================")
print("1. 建立進場與出場訊號並列出")
print(data1[buy].gspc)
print(data1[sell].gspc)

# 2. 此操作策略的累積報酬率為何?
ans = result.cumulative_returns().tail()
print("=====================================")
print(F"2. 累計報酬率：\n{ans*100}%")


# 3. 此操作策略的年化波動率為何?
ans = result.annualized_volatility()
print("=====================================")
print(F"3. 年化波動率：{ans*100}%")

# 4. 此操作策略有比一般策略好嗎?
print("=====================================")
print("4. 從圖三的 Cumulative Returns 來看的話，此策略的收益略遜於一般策略。")
result.plot().show()

1. 建立進場與出場訊號並列出
Date
2017-01-13    2274.639893
2017-01-24    2280.070068
2017-02-03    2297.419922
2017-03-13    2373.469971
2017-03-28    2358.570068
                 ...     
2020-09-15    3401.199951
2020-09-28    3351.600098
2020-10-23    3465.389893
2020-11-03    3369.159912
2020-11-24    3635.409912
Name: gspc, Length: 95, dtype: float64
Date
2017-01-11    2275.320068
2017-01-17    2267.889893
2017-01-31    2278.870117
2017-03-07    2368.389893
2017-03-21    2344.020020
                 ...     
2020-09-18    3319.469971
2020-10-15    3483.340088
2020-10-27    3390.679932
2020-11-19    3581.870117
2020-12-10    3668.100098
Name: gspc, Length: 96, dtype: float64
2. 累計報酬率：
Date
2020-12-09    51.012052
2020-12-10    50.817985
2020-12-11    50.817985
2020-12-14    50.817985
2020-12-15    50.817985
Name: gspc, dtype: float64%
3. 年化波動率：13.816159473892286%
4. 從圖三的 Cumulative Returns 來看的話，此策略的收益略遜於一般策略。


## 4. 迴歸分析

In [10]:
data3 = ffn.get(["^GSPC", "BTC-USD"], start="2017-01-01", end="2020-12-15")
data3['gspc_ret'] = data.gspc.diff() / data.gspc.shift()
data3['btcusd_ret'] = data.btcusd.diff() / data.btcusd.shift()

data3["const"] = 1
x1 = ["const", "btcusd_ret"]
x2 = ["const", "gspc_ret"]

print("******************************************************************************\n")
reg = sm.OLS(data["gspc_ret"], data[x1], missing="drop").fit()
print(reg.summary())

print("******************************************************************************\n")
reg = sm.OLS(data["btcusd_ret"], data[x2], missing="drop").fit()
print(reg.summary())

print("從兩種互相回歸的結果來看，R-squared 都相當的低，只有 1%, 表示回歸的殘差與平均值殘差差不多，兩者互相的預測能力是隨機的，無法預測。")



******************************************************************************

                            OLS Regression Results                            
Dep. Variable:               gspc_ret   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     13.04
Date:                Thu, 31 Dec 2020   Prob (F-statistic):           0.000321
Time:                        15:14:51   Log-Likelihood:                 2918.3
No. Observations:                 995   AIC:                            -5833.
Df Residuals:                     993   BIC:                            -5823.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------

## 5. 加分題: 在不利用套件vectorbt下，以第二題為例
  - 根據你的策略下，計算出進場次數與出場次數，解釋這兩種次數之間的關係 (4%)
  - 假設不考慮任何策略，在一開始便進場，最後出場，計算此策略(buy-and-hold)的年化報酬率與年化波動率為何 (4%)
  - 建立進場與出場訊號之後，在不限制資金的情況下，計算出此策略的最終淨利 (=獲利減去成本) (8%)
  - 請在12/31前上moodle完成(Week14)課堂反映問卷 (2%)

In [11]:
data4 = ffn.get(["^GSPC", "BTC-USD"], start="2017-01-01", end="2020-12-15")

maA = data1.btcusd.rolling(3).mean()
maB = data1.btcusd.rolling(5).mean()

buy = (maA>maB)&(maB.shift()>maA.shift())
sell = (maB>maA)&(maA.shift()>maB.shift())


# 1. 建立進場與出場訊號並列出
print("=====================================")
print("1. 建立進場與出場訊號並列出")
print(len(data4))
print(len(data4[buy].btcusd))
print(len(data4[sell].btcusd))
print(data4[buy].btcusd)
print(data4[sell].btcusd)
print("可以看出進出場是成對的，平均九天會有一次交易，代表著比特幣的交易是可以利用頻繁的週期性價差來獲利。")

result = vbt.Portfolio.from_signals(data1.btcusd, buy, sell, freq='1D')
print("=====================================")
# 2. Buy and Hold
print(F"進場價格：{data4.btcusd[0]}")
print(F"出場價格：{data4.btcusd[-1]}")
acumulated_returns = data4.btcusd[-1] / data4.btcusd[0]
year = len(data4) / 365
annualized_returns = math.pow(1+acumulated_returns, 1/year) - 1
print(F"年化報酬率：{annualized_returns*100} ！")

log_ret = np.log(data4["btcusd"]).diff()
annualized_volatility = (log_ret.std() / log_ret.mean()) * np.sqrt(1/len(data4))
print(F"年化波動率：{annualized_volatility*100}% ！")

profit = data4.btcusd[-1] * 1000000/data4.btcusd[0] - 1000000
print(F"在不考慮資金的情況下投入 100萬美金 可獲得：{profit} USD ！")

1. 建立進場與出場訊號並列出
996
102
101
Date
2017-01-17      907.937988
2017-01-27      919.750000
2017-02-15     1007.479980
2017-03-14     1240.000000
2017-03-29     1026.430054
                  ...     
2020-10-19    11916.334961
2020-11-12    16276.343750
2020-12-02    19201.091797
2020-12-07    19191.630859
2020-12-14    19246.644531
Name: btcusd, Length: 102, dtype: float64
Date
2017-01-26      917.585999
2017-02-10      988.674011
2017-03-08     1150.000000
2017-03-17     1100.229980
2017-04-13     1167.540039
                  ...     
2020-10-16    11358.101562
2020-11-11    15701.339844
2020-11-27    17108.402344
2020-12-04    18699.765625
2020-12-08    18321.144531
Name: btcusd, Length: 101, dtype: float64
可以看出進出場是成對的，平均九天會有一次交易，代表著比特幣的交易是可以利用頻繁的週期性價差來獲利。
進場價格：1043.8399658203125
出場價格：19417.076171875
年化報酬率：197.56414094672454 ！
年化波動率：53.7978469004621% ！
在不考慮資金的情況下投入 100萬美金 可獲得：17601583.391775854 USD ！
